In [ ]:
from datetime import datetime
import os
from pathlib import Path
from PIL import Image
from pillow_heif import register_heif_opener
import re
import yaml
from yaml.loader import SafeLoader

In [ ]:
with open("local.yaml") as f:
    local_data = yaml.load(f, Loader=SafeLoader)

folder = local_data["folder"]
os.chdir(folder)

register_heif_opener()

log_file = open("image_log.txt", 'a')

In [ ]:
def get_file_meta(fname):
    if re.search(r"\.jpg$|\.jpeg$|\.png$|\.bmp$", str(fname), re.IGNORECASE):
        image = Image.open(fname)
        exif_data = image._getexif()

        if exif_data:
            date_original = exif_data.get(36867)
            return date_original
        else:
            log_file.write(str(fname) + " does not have exif data.\n")
    elif re.search(r"\.HEIC$", str(fname), re.IGNORECASE):
        image = Image.open(fname)
        exif_data = image.getexif()

        if exif_data:
            date_original = exif_data[306]
            return date_original
        else:
            log_file.write(str(fname) + " does not have exif data.\n")
    elif re.search(r"\.txt$", str(fname), re.IGNORECASE):
        None
    else:
        log_file.write(str(fname) + " has unknown file format.\n")

    return None
        

In [ ]:
new_name_dict = {}

for fname in Path(folder).glob('*'):
    if fname.is_file():
        try:
            date_original = get_file_meta(fname)

            if date_original:
                date = datetime.strptime(date_original, "%Y:%m:%d %H:%M:%S").strftime('%Y%m%d_%H%M%S')
                name, extension = os.path.splitext(fname)

                # There might be files with same timestamp as creation date. Give them index.
                if (date in new_name_dict.keys()):
                    index = new_name_dict[date] + 1
                    new_name_dict[date] = index
                else:
                    index = 0
                    new_name_dict[date] = index

                new_fname = os.path.join(folder, date + '_' + str(index) + extension)
                log_file.write(str(fname) + " =====> " + str(new_fname))
                os.rename(fname, new_fname)
        except:
            log_file.write(str(fname) + " may be corropted or not an image file.\n")

log_file.close()